# 서비스 이탈예측 데이터
데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)  
x_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv  
y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv  
x_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv  
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv  
데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv")


display(x_train.head())
display(y_train.head())

CustomerId    Surname  CreditScore Geography  Gender  Age  Tenure  \
0    15799217   Zetticci          791   Germany  Female   35       7   
1    15748986    Bischof          705   Germany    Male   42       8   
2    15722004     Hsiung          543    France  Female   31       4   
3    15780966  Pritchard          709    France  Female   32       2   
4    15636731      Ts'ai          714   Germany  Female   36       1   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  
0   52436.20              1          1               0        161051.75  
1  166685.92              2          1               1         55313.51  
2  138317.94              1          0               0         61843.73  
3       0.00              2          0               0        109681.29  
4  101609.01              2          1               1           447.73

CustomerId  Exited
0    15799217       0
1    15748986       0
2    15722004       0
3    15780966       0
4    15636731       0

In [4]:
drop_col = ['CustomerId','Surname']
x_train_drop = x_train.drop(columns = drop_col)
x_test_drop = x_test.drop(columns = drop_col)
display(x_train_drop.head(), x_test_drop.head())

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          791   Germany  Female   35       7   52436.20              1   
1          705   Germany    Male   42       8  166685.92              2   
2          543    France  Female   31       4  138317.94              1   
3          709    France  Female   32       2       0.00              2   
4          714   Germany  Female   36       1  101609.01              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               0        161051.75  
1          1               1         55313.51  
2          0               0         61843.73  
3          0               0        109681.29  
4          1               1           447.73

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          802    France  Female   60       3   92887.06              1   
1          602    France  Female   56       3  115895.22              3   
2          801    France  Female   32       4   75170.54              1   
3          693     Spain  Female   34      10  107556.06              2   
4          592    France  Female   62       5       0.00              1   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               0         39473.63  
1          1               0          4176.17  
2          1               1         37898.50  
3          0               0        154631.35  
4          1               1        100941.57

In [7]:
x_train_dummies = pd.get_dummies(x_train_drop)
y = y_train["Exited"]

x_test_dummies = pd.get_dummies(x_test_drop)
# train과 컬럼 순서 동일하게 하기 (더미화 하면서 순서대로 정렬을 이미 하기 때문에 오류가 난다면 해당 컬럼이 누락된것)
x_test_dummies = x_test_dummies[x_train_dummies.columns]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_validation, Y_train, Y_validation = train_test_split(x_train_dummies, y, test_size = 0.33, random_state = 42)

rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, Y_train)

RandomForestClassifier(random_state=23)

In [12]:
import sklearn.metrics
from sklearn.metrics import accuracy_score , f1_score, recall_score, roc_auc_score ,precision_score

predict_train_label = rf.predict(X_train) # 각각 모델의 최종 예측값 출력 -> 분류 모델 : 0,1,2,3 등 가장 확률 높은 클래스 출력
predict_train_proba = rf.predict_proba(X_train)[:,1] # 분류 모델이 각 클래스에 대한 확률을 출력

predict_validation_label = rf.predict(X_validation)
predict_validation_prob = rf.predict_proba(X_validation)[:,1]

In [17]:
# 문제에서 묻는 것에 따라 모델 성능 확인하기
# 정확도 (accuracy) , f1_score , recall , precision -> model.predict로 결과뽑기
# auc , 확률이라는 표현있으면 model.predict_proba로 결과뽑고 첫번째 행의 값을 가져오기 model.predict_proba()[:,1]
print('train accuracy :', accuracy_score(Y_train,predict_train_label))
print('validation accuracy :', accuracy_score(Y_validation,predict_validation_label))
print('\n')

print('train f1_score :', f1_score(Y_train,predict_train_label))
print('validation accuracy :', f1_score(Y_validation,predict_validation_label))

print('\n')
print('train recall_score :', recall_score(Y_train,predict_train_label))
print('validation recall_score :', recall_score(Y_validation,predict_validation_label))

print('\n')
print('train precision_score :', precision_score(Y_train,predict_train_label))
print('validation precision_score :', precision_score(Y_validation,predict_validation_label))

print('\n')
print('train auc :', roc_auc_score(Y_train,predict_train_proba))
print('validation auc :', roc_auc_score(Y_validation,predict_validation_prob))

train accuracy : 1.0
validation accuracy : 0.8652680652680653


train f1_score : 1.0
validation accuracy : 0.5912305516265912


train recall_score : 1.0
validation recall_score : 0.4543478260869565


train precision_score : 1.0
validation precision_score : 0.8461538461538461


train auc : 1.0
validation auc : 0.8497613211198555


In [ ]:
# test데이터 마찬가지 위와 같은 방식
predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:,1]